<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Exploring-github-APIs-possibilities" data-toc-modified-id="Exploring-github-APIs-possibilities-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Exploring github APIs possibilities</a></span></li></ul></div>

This notebook contains code that was used to explore
the github API in view of making `hubcap`.

I give no assurances as to if anything still works here.

In [75]:
from guide import print_attrs_info

# Objects

In [6]:
from hubcap import GithubReader

gr = GithubReader()
gr._github.get_user(), gr.src

(AuthenticatedUser(login=None), AuthenticatedUser(login=None))

In [7]:
gr = GithubReader('i2mint')
gr.src

NamedUser(login="i2mint")

In [30]:
from hubcap import hubcap

repositories = hubcap('thorwhalen')
'hubcap' in repositories
hubcap_repo = repositories['hubcap']

repo = hubcap('thorwhalen/hubcap')
'master' in repo
master_files = repo['master']

files = hubcap('thorwhalen/hubcap/master')
'/README.md' in files
'/hubcap/' in files
hubcap_files = files['/hubcap/']
'/hubcap/base.py' in hubcap_files


discussions = hubcap('thorwhalen/hubcap/discussions')
2 in discussions
discussion = discussions[2]
discussion['title']


False

In [13]:
from hubcap import hubcap
issues = hubcap('thorwhalen/hubcap/issues')
3 in issues  # there's a "number 3" issue
issue = issues[3]
issue.title
issue.body
issue.comments



1

In [3]:
list(issues)

[]

In [57]:
hubcap_files_ = hubcap('thorwhalen/hubcap/master/hubcap/')
'/hubcap/base.py' in hubcap_files_
# s = hubcap('thorwhalen/hubcap/tree/b21958421d2eddfa9340f710e933a454938d4dcb')



'Root interface of hubcap'

In [50]:
str.st

github.Repository.Repository

In [15]:
list(s)

['/.gitattributes',
 '/.github/',
 '/.gitignore',
 '/LICENSE',
 '/README.md',
 '/docsrc/',
 '/hubcap/',
 '/misc/',
 '/setup.cfg',
 '/setup.py']

In [18]:
print(s['/hubcap/base.py'].decode())

"""Base objects"""
from functools import cached_property, partial
from operator import attrgetter
from dol import KvReader
from github import GithubException, Github
import github

from dol.util import format_invocation

NotSet = github.GithubObject.NotSet


def decoded_contents(content_file):
    return content_file.decoded_content
    # from base64 import b64decode
    # return b64decode(content_file.content).decode()


# TODO: Enable user parametrization of how to find token
#  Use: config2py
def find_github_token():
    import os

    return os.environ.get("HUBCAP_GITHUB_TOKEN", None) or os.environ.get(
        "GITHUB_TOKEN", None
    )


def find_user_name():
    import os

    return os.environ.get("HUBCAP_GITHUB_TOKEN", None) or os.environ.get(
        "GITHUB_USERNAME", None
    )


# TODO: use signature arithmetic
# @kv_decorator
class GithubReader(KvReader):
    """
    a Store that can access a GitHub account
    """

    def __init__(
        self,
        account_name: st

# Exploring github APIs possibilities

In [1]:
import os
from github import Github

ddir = lambda o: [a for a in dir(o) if not a.startswith('_')]
print(*ddir(Github), sep=', ')

FIX_REPO_GET_GIT_REF, create_from_raw_data, dump, get_app, get_emojis, get_events, get_gist, get_gists, get_gitignore_template, get_gitignore_templates, get_hook, get_hooks, get_license, get_licenses, get_oauth_application, get_organization, get_organizations, get_project, get_project_column, get_rate_limit, get_repo, get_repos, get_user, get_user_by_id, get_users, load, oauth_scopes, per_page, rate_limiting, rate_limiting_resettime, render_markdown, search_code, search_commits, search_issues, search_repositories, search_topics, search_users


In [2]:
get_methods = [a for a in dir(Github) if a.startswith('get_')]
get_names = [a[4:] for a in get_methods]
get_names

['app',
 'emojis',
 'events',
 'gist',
 'gists',
 'gitignore_template',
 'gitignore_templates',
 'hook',
 'hooks',
 'license',
 'licenses',
 'oauth_application',
 'organization',
 'organizations',
 'project',
 'project_column',
 'rate_limit',
 'repo',
 'repos',
 'user',
 'user_by_id',
 'users']

In [72]:
print_attrs_info(Github)

----- Methods -----
         get_rate_limit: 
            get_license: key
           get_licenses: 
             get_events: 
               get_user: login
         get_user_by_id: user_id
              get_users: since
       get_organization: login
      get_organizations: since
               get_repo: full_name_or_id, lazy
              get_repos: since, visibility
            get_project: id
     get_project_column: id
               get_gist: id
              get_gists: since
    search_repositories: query, sort, order, qualifiers
           search_users: query, sort, order, qualifiers
          search_issues: query, sort, order, qualifiers
            search_code: query, sort, order, highlight, qualifiers
         search_commits: query, sort, order, qualifiers
          search_topics: query, qualifiers
        render_markdown: text, context
               get_hook: name
              get_hooks: 
get_gitignore_templates: 
 get_gitignore_template: name
             get_emojis: 


In [3]:
g = Github()

In [4]:
t = g.get_user('thorwhalen')
t

NamedUser(login="thorwhalen")

In [5]:
type(t).mro()

[github.NamedUser.NamedUser,
 github.GithubObject.CompletableGithubObject,
 github.GithubObject.GithubObject,
 object]

In [8]:
from hubcap.base import PaginatedListDol
p = PaginatedListDol(t.get_followers())
list(p)

[NamedUser(login="thejeshgn"),
 NamedUser(login="Rokotyan"),
 NamedUser(login="yz-"),
 NamedUser(login="Magicianred"),
 NamedUser(login="iDoka"),
 NamedUser(login="leb9212"),
 NamedUser(login="cavart28"),
 NamedUser(login="izcet"),
 NamedUser(login="andeaseme"),
 NamedUser(login="jakka351"),
 NamedUser(login="zfeng10")]

In [120]:
print_attrs_info(t)

----- Methods -----
                get__repr__: params
                 get_events: 
              get_followers: 
              get_following: 
                  get_gists: since
                   get_keys: 
get_organization_membership: org
                   get_orgs: 
               get_projects: state
          get_public_events: 
 get_public_received_events: 
        get_received_events: 
                   get_repo: name
                  get_repos: type, sort, direction
                get_starred: 
          get_subscriptions: 
                get_watched: 
           has_in_following: following
      setCheckAfterInitFlag: flag
                     update: additional_headers
------ Props ------
CHECK_AFTER_INIT_FLAG
avatar_url
bio
blog
collaborators
company
contributions
created_at
disk_usage
email
etag
events_url
followers
followers_url
following
following_url
gists_url
gravatar_id
hireable
html_url
id
invitation_teams_url
inviter
last_modified
location
login
name
node_id
o

In [122]:
r = t.get_repos()
print_attrs_info(r)

----- Methods -----
  get_page: page
------ Props ------
reversed
totalCount


In [142]:
from hubcap.base import PaginatedListDol

rr = PaginatedListDol(r)
repositories = list(rr)
repositories

[Repository(full_name="thorwhalen/agen"),
 Repository(full_name="thorwhalen/aix"),
 Repository(full_name="thorwhalen/analysis_dashboard"),
 Repository(full_name="thorwhalen/babe"),
 Repository(full_name="thorwhalen/bc"),
 Repository(full_name="thorwhalen/boltons"),
 Repository(full_name="thorwhalen/brand"),
 Repository(full_name="thorwhalen/bumph"),
 Repository(full_name="thorwhalen/cachetools"),
 Repository(full_name="thorwhalen/codify"),
 Repository(full_name="thorwhalen/cpython"),
 Repository(full_name="thorwhalen/cult"),
 Repository(full_name="thorwhalen/drug_lifecourse"),
 Repository(full_name="thorwhalen/elections"),
 Repository(full_name="thorwhalen/equate"),
 Repository(full_name="thorwhalen/esimport"),
 Repository(full_name="thorwhalen/essays"),
 Repository(full_name="thorwhalen/fbconsole"),
 Repository(full_name="thorwhalen/garble"),
 Repository(full_name="thorwhalen/graze"),
 Repository(full_name="thorwhalen/grub"),
 Repository(full_name="thorwhalen/guide"),
 Repository(full

In [171]:
s = g.search_repositories('sound recognition')
s.totalCount

566

In [170]:
list(s)

[Repository(full_name="carlosmwh1985/sound_recognition_ml"),
 Repository(full_name="otosense/viable"),
 Repository(full_name="ChaimTeitelbaum/Project_3_ML_sound_recognition"),
 Repository(full_name="MahmoodAbuAwwad/Sound_Detection_ML"),
 Repository(full_name="saurabh23897/FaultyTileSoundRecognition_PythonML"),
 Repository(full_name="tayciryahmed/ml-challenges"),
 Repository(full_name="chelakis/---Automatic-Speaker-Recognition-using-ML"),
 Repository(full_name="jogomez97/BirdMLClassification"),
 Repository(full_name="snpushpi/Speech_Emotion_recognition_Project"),
 Repository(full_name="doans/Underwater-Acoustic-Target-Classification-Based-on-Dense-Convolutional-Neural-Network"),
 Repository(full_name="erikd234/BikeSentry"),
 Repository(full_name="AnbuKumar-maker/Machine-Learning---Sensor-Data-Analytics")]

In [158]:
repo = repositories[19]
repo

Repository(full_name="thorwhalen/graze")

In [145]:
print_attrs_info(repo)

----- Methods -----
            add_to_collaborators: collaborator, permission
                         compare: base, head
                create_check_run: name, head_sha, details_url, external_id, status, started_at, conclusion, completed_at, output, actions
              create_check_suite: head_sha
               create_deployment: ref, task, auto_merge, required_contexts, payload, environment, description, transient_environment, production_environment
                     create_file: path, message, content, branch, committer, author
                     create_fork: organization
                 create_git_blob: content, encoding
               create_git_commit: message, tree, parents, author, committer
                  create_git_ref: ref, sha
              create_git_release: tag, name, message, draft, prerelease, target_commitish
                  create_git_tag: tag, message, object, type, tagger
      create_git_tag_and_release: tag, tag_message, release_name, release_mes

In [160]:
w = PaginatedListDol(repo.get_commits())
len(w)

24

In [165]:
w = PaginatedListDol(g.get_repo('i2mint/dol').get_commits())
print(f"{len(w)=}")
ww = list(w.paginated_list)
print(f"{len(ww)=}")
ww

len(w)=168
len(ww)=168


[Commit(sha="5b3c21c9f44f0195bc67c51aa2888018b4566e48"),
 Commit(sha="eb4dcd22e1f62a696a133438a43f33a959d96a65"),
 Commit(sha="9ae5bf466d78c8882bc2271a5f60965a52634c36"),
 Commit(sha="0de80581df1ac6fd480d64d3dd13f85b2c61239f"),
 Commit(sha="be0dab5ca022a2a791bab336ce9cfc6807bd9ba7"),
 Commit(sha="aa8e9ab7ef800f2b82fcbdee4ec3903596d94780"),
 Commit(sha="c44980ee350f3c45ba8fedb01dbc609f7b8db457"),
 Commit(sha="80441529c1482c5603b9f4f9d68b19ec9d2c101e"),
 Commit(sha="f26e680acce533063406a8a95bc74e9268d1b970"),
 Commit(sha="dd62ceee69d1b2767e2e6196e0570948ed2c5d13"),
 Commit(sha="34fc90886142dbae3f394a0ae739b5ae4e729fd7"),
 Commit(sha="c043a2137341db8583c2b5e685d0a892658849d2"),
 Commit(sha="a9e1610867192075efd5a5a24f2c58a5e16ac1eb"),
 Commit(sha="771e7aa100beef8e472e4a216b3e66cb381243ac"),
 Commit(sha="da53550ae40c495ff4f35585d616f4b7a359b1e6"),
 Commit(sha="7ba4e6e4ab2eb0508db4a8f4552cf66d67c09599"),
 Commit(sha="854bc21151cbf9484df311777ee32f1548f1f55e"),
 Commit(sha="2be57daa5aeb617b34

In [157]:
repo = g.get_repo('i2mint/dol')
repo

Repository(full_name="i2mint/dol")

In [156]:
w = PaginatedListDol(repo.get_collaborators())
list(w)

GithubException: 403 {"message": "Must have push access to view repository collaborators.", "documentation_url": "https://docs.github.com/rest/reference/repos#list-repository-collaborators"}

In [125]:
type(r).mro()

[github.PaginatedList.PaginatedList,
 github.PaginatedList.PaginatedListBase,
 object]

In [121]:
p = t.get_gists()
print_attrs_info(p)

----- Methods -----
  get_page: page
------ Props ------
reversed
totalCount


In [112]:
p.totalCount

18

In [79]:
for x in type(t).mro():
    print(f'******** {x} *************')
    print_attrs_info(x)

******** <class 'github.NamedUser.NamedUser'> *************
----- Methods -----
                 get_events: 
              get_followers: 
              get_following: 
                  get_gists: since
                   get_keys: 
                   get_orgs: 
               get_projects: state
          get_public_events: 
 get_public_received_events: 
        get_received_events: 
                   get_repo: name
                  get_repos: type, sort, direction
                get_starred: 
          get_subscriptions: 
                get_watched: 
           has_in_following: following
get_organization_membership: org
------ Props ------
node_id
twitter_username
avatar_url
bio
blog
collaborators
company
contributions
created_at
disk_usage
email
events_url
followers
followers_url
following
following_url
gists_url
gravatar_id
hireable
html_url
id
invitation_teams_url
inviter
location
login
name
organizations_url
owned_private_repos
permissions
plan
private_gists
public_gists
p

In [73]:
print_attrs_info(t)

ValueError: max() arg is an empty sequence

In [27]:
tt = g.get_repo()

[Event(type="PushEvent", id="19056802526"),
 Event(type="PushEvent", id="19056802517"),
 Event(type="PushEvent", id="19056802512"),
 Event(type="PushEvent", id="19056802520"),
 Event(type="IssueCommentEvent", id="19056802516"),
 Event(type="WatchEvent", id="19056802522"),
 Event(type="PublicEvent", id="19056802510"),
 Event(type="PushEvent", id="19056802502"),
 Event(type="PushEvent", id="19056802499"),
 Event(type="PushEvent", id="19056802487"),
 Event(type="PushEvent", id="19056802486"),
 Event(type="PushEvent", id="19056802488"),
 Event(type="DeleteEvent", id="19056802498"),
 Event(type="PushEvent", id="19056802469"),
 Event(type="PushEvent", id="19056802482"),
 Event(type="IssuesEvent", id="19056802479"),
 Event(type="PullRequestEvent", id="19056802474"),
 Event(type="CreateEvent", id="19056802458"),
 Event(type="PullRequestEvent", id="19056802448"),
 Event(type="PushEvent", id="19056802442"),
 Event(type="PullRequestReviewEvent", id="19056802456"),
 Event(type="PushEvent", id="190

In [14]:
g = Github(os.getenv('GITHUB_TOKEN'))
t = g.get_emojis()
t

{'+1': 'https://github.githubassets.com/images/icons/emoji/unicode/1f44d.png?v8',
 '-1': 'https://github.githubassets.com/images/icons/emoji/unicode/1f44e.png?v8',
 '100': 'https://github.githubassets.com/images/icons/emoji/unicode/1f4af.png?v8',
 '1234': 'https://github.githubassets.com/images/icons/emoji/unicode/1f522.png?v8',
 '1st_place_medal': 'https://github.githubassets.com/images/icons/emoji/unicode/1f947.png?v8',
 '2nd_place_medal': 'https://github.githubassets.com/images/icons/emoji/unicode/1f948.png?v8',
 '3rd_place_medal': 'https://github.githubassets.com/images/icons/emoji/unicode/1f949.png?v8',
 '8ball': 'https://github.githubassets.com/images/icons/emoji/unicode/1f3b1.png?v8',
 'a': 'https://github.githubassets.com/images/icons/emoji/unicode/1f170.png?v8',
 'ab': 'https://github.githubassets.com/images/icons/emoji/unicode/1f18e.png?v8',
 'abacus': 'https://github.githubassets.com/images/icons/emoji/unicode/1f9ee.png?v8',
 'abc': 'https://github.githubassets.com/images/ic